In [1]:
# import libraries 
import numpy as np
import plotly.express as px
import pandas as pd
import statistics

In [2]:
insurance = pd.read_csv('insurance.csv')

In [0]:
# NOTE: linreg provided a 77% prediction rate. GridSearch optimization did not aid in increasing this number.
# Conclusion: use a different model

In [3]:
def smokerToBinary(cond):
    if cond == "yes":
        return 1
    elif cond == "no":
        return 0

insurance["binary_smoker"] = insurance["smoker"].apply(smokerToBinary)

In [4]:
def sexToBinary(sex):
    if sex == "male":
        return 1
    elif sex == "female":
        return 0

insurance["binary_sex"] = insurance["sex"].apply(sexToBinary)

In [5]:
enc_region = pd.get_dummies(insurance["region"])
insurance_cleaned = pd.concat((insurance, enc_region), axis=1)

In [6]:
insurance_cleaned.drop("smoker", axis=1, inplace=True)
insurance_cleaned.drop("sex", axis=1, inplace=True)
insurance_cleaned.drop("region", axis=1, inplace=True)

In [7]:
insurance_cleaned.head()

,age,bmi,children,charges,binary_smoker,binary_sex,northeast,northwest,southeast,southwest
0,19,27.900,0,16884.92400,1,0,0,0,0,1
1,18,33.770,1,1725.55230,0,1,0,0,1,0
2,28,33.000,3,4449.46200,0,1,0,0,1,0
3,33,22.705,0,21984.47061,0,1,0,1,0,0
4,32,28.880,0,3866.85520,0,1,0,1,0,0


In [8]:
target = insurance_cleaned["charges"]

In [9]:
input_columns = insurance_cleaned.loc[:, insurance_cleaned.columns != "charges"]

In [53]:
import sklearn
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier as KNN (will not work the dataset that we have)
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(input_columns, target, test_size=0.2)

In [83]:
# {'copy_X': True,
#  'fit_intercept': True,
#  'n_jobs': 2,
#  'normalize': False,
#  'positive': True}


linr = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=2, normalize=False, positive=True)

In [84]:
linr.fit(x_train, y_train)

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.



LinearRegression(n_jobs=2, normalize=False, positive=True)

In [85]:
y_hat = linr.predict(x_test)

In [57]:
MAE = np.mean(abs(y_test - y_hat))
MAE

4203.677062890177

In [86]:
linr.score(x_test, y_test)

# score()

#1. Takes x_test & makes y_hat
#2. Takes new y_hat and calculates R-2 Coefficient w/ y_test



0.777505964407186

In [77]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_hat)

4203.677062890172

In [63]:
import matplotlib.pyplot as plt

# plt.figure(figsize=(10,5))
# plt.scatter(x_test, y_hat)

# plt.scatter(x_test,y_test, c='orange')  
# plt.xlabel('x', fontsize = 20) 
# plt.ylabel('y', fontsize = 20)
# plt.title('Generated Data - Test')
# plt.grid('on')
# plt.show()

x_test

,age,bmi,children,binary_smoker,binary_sex,northeast,northwest,southeast,southwest
764,45,25.175,2,0,0,1,0,0,0
929,41,34.210,1,0,1,0,0,1,0
674,44,43.890,2,1,0,0,0,1,0
825,64,31.825,2,0,0,1,0,0,0
169,27,18.905,3,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
79,41,32.965,0,0,0,0,1,0,0
708,31,30.495,3,0,0,1,0,0,0
232,19,17.800,0,0,0,0,0,0,1
1289,44,34.320,1,0,1,0,0,1,0


In [71]:
results_df = pd.DataFrame({"x": x_test["age"], "y": y_hat})
results_df.head()

,x,y
764,45,9295.829586
929,41,9505.510395
674,44,37738.118451
825,64,16416.385153
169,27,2758.701780


In [74]:
import plotly.express as px

px.scatter(results_df, x = "x", y = "y", trendline = "ols")

In [81]:
from sklearn.model_selection import GridSearchCV

parameters = {
    "copy_X": [True, False],
    "fit_intercept": [True, False],
    "n_jobs": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    "normalize": [True, False],
    "positive": [True, False]
}

optimizer = GridSearchCV(LinearRegression(), param_grid = parameters)
optimizer.fit(x_train, y_train)

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)



/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)



/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

/projects/200e8042-15d9-45b5-b665-c79cbc8761e9/miniconda3/envs/ds_env/lib/python3.8/

In [82]:
optimizer.best_params_

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': 2,
 'normalize': False,
 'positive': True}